In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/03 16:25:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [14]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
""")

In [12]:
df_green_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|               45|
|2020-01-01 00:00:00|  17|            195.03|                9|
|2020-01-01 00:00:00|  18|               7.8|                1|
|2020-01-01 00:00:00|  22|              15.8|                1|
|2020-01-01 00:00:00|  24|              87.6|                3|
|2020-01-01 00:00:00|  25|             531.0|               26|
|2020-01-01 00:00:00|  29|              61.3|                1|
|2020-01-01 00:00:00|  32| 68.94999999999999|                2|
|2020-01-01 00:00:00|  33|            317.27|               11|
|2020-01-01 00:00:00|  35|            129.96|                5|
|2020-01-01 00:00:00|  36|295.34000000000003|               11|
|2020-01-01 00:00:00|  37|            175.67|                6|
|2020-01-01 00:00:00|  38| 98.7899999999

In [20]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [17]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [18]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM 
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
""")

In [21]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [31]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_of_records', 'green_number_of_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_of_records', 'yellow_number_of_records')

In [33]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [35]:
df_join \
    .repartition(20) \
    .write.parquet('data/report/revenue/total', mode='overwrite')

In [28]:
df_join.show()

+-------------------+----+------------+-----------------------+------------------+------------------------+
|               hour|zone|green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|
+-------------------+----+------------+-----------------------+------------------+------------------------+
|2020-01-01 00:00:00|   3|        null|                   null|              25.0|                       1|
|2020-01-01 01:00:00|  17|      598.15|                     18|            464.51|                      18|
|2020-01-01 01:00:00| 107|        null|                   null| 9994.480000000021|                     583|
|2020-01-01 01:00:00| 162|        null|                   null| 5736.690000000002|                     298|
|2020-01-01 02:00:00| 234|        null|                   null| 6759.990000000009|                     370|
|2020-01-01 03:00:00| 170|        null|                   null|            4632.0|                     257|
|2020-01-01 04:00:00|  22|  

In [36]:
df_join = spark.read.parquet('data/report/revenue/total')

In [37]:
df_join.show()

+-------------------+----+-----------------+-----------------------+------------------+------------------------+
|               hour|zone|     green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|
+-------------------+----+-----------------+-----------------------+------------------+------------------------+
|2020-08-27 16:00:00| 197|848.9999999999998|                     17|            398.18|                       8|
|2020-03-03 18:00:00|  36|             null|                   null|              16.8|                       1|
|2020-05-02 05:00:00| 234|             null|                   null|               9.3|                       1|
|2021-04-07 02:00:00| 164|             null|                   null|50.260000000000005|                       2|
|2020-08-07 01:00:00| 228|             null|                   null|             382.0|                       5|
|2020-07-11 21:00:00|  79|             null|                   null|            330.24|         

In [40]:
from pyspark.sql import types

In [41]:
zone_schema = types.StructType([
    types.StructField("LocationID",types.IntegerType(),True),
    types.StructField("Borough",types.StringType(),True),
    types.StructField("Zone",types.StringType(),True),
    types.StructField("service_zone",types.StringType(),True)
])

In [43]:
df_zones = spark.read \
    .schema(zone_schema) \
    .option("header", "true") \
    .csv("taxi+_zone_lookup.csv")

In [44]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [45]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [49]:
df_result.drop('LocationID', 'zone').write.parquet('data/tmp/revenue-zones')